# Analyze Issue Script and Workflow Test

This notebook tests the `analyze_issue.py` script and the corresponding GitHub Actions workflow. The tests include validating the OpenAI API key, GitHub token, and running the script to check for errors.

In [1]:
import os
import json
import openai
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_REPO = os.getenv("GITHUB_REPO", "https://github.com/pkuppens/my_chat_gpt")

# Set up OpenAI
openai.api_key = OPENAI_API_KEY

## Validate OpenAI API Key and Permissions

In [2]:
openai_client = openai.OpenAI()

models = openai_client.get(path="models", cast_to=str)


In [3]:
models_dict= json.loads(models)

In [4]:

models_ids = [
    model["id"]
    for model in models_dict.get("data", [])
]

print(models_ids)

['gpt-4.5-preview', 'omni-moderation-2024-09-26', 'gpt-4.5-preview-2025-02-27', 'gpt-4o-mini-audio-preview-2024-12-17', 'dall-e-3', 'dall-e-2', 'gpt-4o-audio-preview-2024-10-01', 'gpt-4o-audio-preview', 'gpt-4o-mini-realtime-preview-2024-12-17', 'gpt-4o-mini-realtime-preview', 'o1-mini-2024-09-12', 'o1-preview-2024-09-12', 'o1-mini', 'o1-preview', 'gpt-4o-mini-audio-preview', 'whisper-1', 'gpt-4o-2024-05-13', 'gpt-4o-realtime-preview-2024-10-01', 'babbage-002', 'chatgpt-4o-latest', 'tts-1-hd-1106', 'gpt-4o-audio-preview-2024-12-17', 'gpt-4', 'gpt-4o-2024-11-20', 'tts-1-hd', 'gpt-4o-mini-2024-07-18', 'gpt-4o-mini', 'text-embedding-3-large', 'tts-1', 'tts-1-1106', 'gpt-4o-2024-08-06', 'davinci-002', 'gpt-4o', 'gpt-3.5-turbo-1106', 'gpt-4-turbo', 'gpt-3.5-turbo-instruct', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo-0125', 'gpt-4-turbo-preview', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-3.5-turbo', 'gpt-4-turbo-2024-04-09', 'gpt-4o-realtime-preview', 'gpt-3.5-turbo-16k', 'gpt-4-0125-pre

## Validate GitHub Token and Permissions

In [5]:
def validate_github_token():
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    url = "https://api.github.com/user"
    response = requests.get(url, headers=headers)
    return response.status_code == 200

if validate_github_token():
    print("GitHub token is valid and has sufficient permissions.")
else:
    print("Invalid GitHub token or insufficient permissions.")

GitHub token is valid and has sufficient permissions.


In [6]:
assert GITHUB_REPO == 'https://github.com/pkuppens/my_chat_gpt', "Please set the GITHUB_REPO environment variable to your repository URL."

GITHUB_API_URL = GITHUB_REPO.replace("https://github.com", "https://api.github.com/repos")


In [7]:
def get_github_issue(issue_number):
    url = f"{GITHUB_API_URL}/issues/{issue_number}"
    response = requests.get(url)
    response.raise_for_status()
    return response

response = get_github_issue(21)
print(response.text)

{"url":"https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21","repository_url":"https://api.github.com/repos/pkuppens/my_chat_gpt","labels_url":"https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21/labels{/name}","comments_url":"https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21/comments","events_url":"https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21/events","html_url":"https://github.com/pkuppens/my_chat_gpt/issues/21","id":2865836665,"node_id":"I_kwDOJeQqcc6q0TJ5","number":21,"title":"Idea: Type of Thought to improve answers.","user":{"login":"pkuppens","id":8671392,"node_id":"MDQ6VXNlcjg2NzEzOTI=","avatar_url":"https://avatars.githubusercontent.com/u/8671392?v=4","gravatar_id":"","url":"https://api.github.com/users/pkuppens","html_url":"https://github.com/pkuppens","followers_url":"https://api.github.com/users/pkuppens/followers","following_url":"https://api.github.com/users/pkuppens/following{/other_user}","gists_url":"https://api.github.com/users/

In [8]:
issue_data = response.json()
title = issue_data["title"]
description = issue_data["body"]
print(f"Title: {title}")
print(f"Description: {description}")

Title: Idea: Type of Thought to improve answers.
Description: Task: Create sample implementations for the following idea on guiding prompts and issue analysis.

For instance:
* memory integration: include a design markdown document in the repo that can be read and used in the analysis
* goal setting: issues that require an implementation to be completed could use definition of done as acceptance criteria.

The idea to work from:
Guiding prompts with a hint of Thought type might improve answers.

This idea was inspired by: [Hugging Face Agents Course](https://huggingface.co/learn/agents-course/unit1/thoughts)

| Type of Thought | Example |
|--------|--------|
| Planning | “I need to break this task into three steps: 1) gather data, 2) analyze trends, 3) generate report” |
| Analysis | “Based on the error message, the issue appears to be with the database connection parameters” |
| Decision Making | “Given the user’s budget constraints, I should recommend the mid-tier option” |
| Problem

In [9]:
issue_data

{'url': 'https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21',
 'repository_url': 'https://api.github.com/repos/pkuppens/my_chat_gpt',
 'labels_url': 'https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21/labels{/name}',
 'comments_url': 'https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21/comments',
 'events_url': 'https://api.github.com/repos/pkuppens/my_chat_gpt/issues/21/events',
 'html_url': 'https://github.com/pkuppens/my_chat_gpt/issues/21',
 'id': 2865836665,
 'node_id': 'I_kwDOJeQqcc6q0TJ5',
 'number': 21,
 'title': 'Idea: Type of Thought to improve answers.',
 'user': {'login': 'pkuppens',
  'id': 8671392,
  'node_id': 'MDQ6VXNlcjg2NzEzOTI=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/8671392?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/pkuppens',
  'html_url': 'https://github.com/pkuppens',
  'followers_url': 'https://api.github.com/users/pkuppens/followers',
  'following_url': 'https://api.github.com/users/pkuppens/fo

In [10]:
from SuperPrompt import load_analyze_issue_prompt, ISSUE_TYPES, PRIORITY_LEVELS

In [11]:
placeholders = {
    "title": title,
    "description": description,
    "issue_types": ', '.join(ISSUE_TYPES),
    "priority_levels": ', '.join(PRIORITY_LEVELS),
    "issue_title": issue_data['title'],
    "issue_body": issue_data['body'],
    "issue_data": issue_data
}
placeholders

{'title': 'Idea: Type of Thought to improve answers.',
 'description': "Task: Create sample implementations for the following idea on guiding prompts and issue analysis.\n\nFor instance:\n* memory integration: include a design markdown document in the repo that can be read and used in the analysis\n* goal setting: issues that require an implementation to be completed could use definition of done as acceptance criteria.\n\nThe idea to work from:\nGuiding prompts with a hint of Thought type might improve answers.\n\nThis idea was inspired by: [Hugging Face Agents Course](https://huggingface.co/learn/agents-course/unit1/thoughts)\n\n| Type of Thought | Example |\n|--------|--------|\n| Planning | “I need to break this task into three steps: 1) gather data, 2) analyze trends, 3) generate report” |\n| Analysis | “Based on the error message, the issue appears to be with the database connection parameters” |\n| Decision Making | “Given the user’s budget constraints, I should recommend the mid

In [12]:
prompt = load_analyze_issue_prompt(placeholders)
prompt

"    Analyze and review this GitHub issue and provide the following:\n    1. Issue Type (select one): Epic, Change Request, Bug Fix, Task, Question\n    2. Priority (select one): Critical, High, Medium, Low\n    3. Estimated complexity (select one): Simple, Moderate, Complex\n    4. Feedback:\n        1. the Title is clear and concise, and matches the description - or suggests improvements\n        2. the Description is unambiguous, detailed, and provides necessary context - or suggests improvements\n        3. the Description is SMART (Specific, Measurable, Achievable, Relevant, Time-bound) - or suggests improvements\n        4. any additional comments or suggestions for the issue\n    5. Analysis:\n        Depending on the issue type, provide a detailed analysis of the issue, including:\n        1. Summarize the key points of the issue (for a Bug Fix, this might be a root cause analysis)\n        2. Identify any potential blockers, dependencies, ambiguities, risks, conflicts (a Chang

## Proof of Concept: Example Review Results on a File

In [ ]:
from smolagents import tool

@tool
def review_file(file_path: str, style_guide: str) -> str:
    """Review a file for a certain style guide.

    Args:
        file_path: The path to the file to review.
        style_guide: The style guide to review against.

    Returns:
        str: The review results.
    """
    with open(file_path, 'r') as file:
        content = file.read()

    # Placeholder for actual review logic
    review_results = f"Reviewing {file_path} against {style_guide}...\n"

    # Here we would run the actual review logic, passing content and style_guide to an LLM
    review_results += "No issues found."

    return review_results

example_file_path = "notebooks/review_test_script.py"
example_style_guide = "PEP 8"

review_results = review_file(example_file_path, example_style_guide)
print(review_results)

Reviewing notebooks/review_test_script.py against PEP 8...
No issues found.
